In [1]:
import dateutil.relativedelta as relativedelta
import dateutil.rrule as rrule
import datetime

import numpy as np
import pandas as pd
import pandas_profiling

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

plt.rc('font', family='Malgun Gothic')


Bad key "text.kerning_factor" on line 4 in
C:\Users\User\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


# 카드매출

In [4]:
rs_card_data = pd.read_csv('../../data/bigcontest/02_카드매출데이터(신한카드)/CARD_SPENDING_RESIDENT/CARD_SPENDING_RESIDENT.txt', sep='\t')

In [5]:
rs_card_data.head()

,STD_DD,GU_CD,DONG_CD,MCT_CAT_CD,SEX_CD,AGE_CD,USE_CNT,USE_AMT
0,20190201,110,517,10,F,25,11,595
1,20190201,110,517,10,F,50,5,243
2,20190201,110,517,10,F,55,5,27
3,20190201,110,517,10,M,20,11,243
4,20190201,110,517,10,M,25,11,433


## 전처리
### mct_cat_cd 한글로 변환

In [6]:
mct_dict = { 10:'숙박', 20:'레저용품', 21:'레저업소', 22:'문화취미',
 30:'가구', 31:'전기', 32:'주방용구', 33:'연료판매',
 34:'광학제품', 35:'가전', 40:'유통업', 42:'의복',
 43:'직물', 44:'신변잡화', 50:'서적문구', 52:'사무통신',
 60:'자동차판매', 62:'자동차정비', 70:'의료기관', 71:'보건위생',
 80:'요식업소', 81:'음료식품', 92:'수리서비스' }

In [7]:
rs_card_data.MCT_CAT_CD = [mct_dict[cnt] for cnt in rs_card_data.MCT_CAT_CD]

In [8]:
category_integration_dict = {'가구': {'통합': '가구'}, '가전': {'통합': '전기가전'}, '광학제품': {'통합': '광학제품'},
 '레저업소': {'통합': '레저업소'}, '레저용품': {'통합': '레저용품'}, '문화취미': {'통합': '문화취미'}, '보건위생': {'통합': '보건위생'},
 '사무통신': {'통합': '사무통신'}, '서적문구': {'통합': '서적문구'}, '수리서비스': {'통합': '수리서비스'}, '숙박': {'통합': '숙박'},
 '신변잡화': {'통합': '신변잡화'}, '연료판매': {'통합': '연료판매'}, '요식업소': {'통합': '요식업소'}, '유통업': {'통합': '유통업'},
 '음료식품': {'통합': '음료식품'}, '의료기관': {'통합': '의료기관'}, '의복': {'통합': '의복'}, '자동차정비': {'통합': '자동차정비'},
 '자동차판매': {'통합': '자동차판매'}, '전기': {'통합': '전기가전'}, '주방용구': {'통합': '주방용구'}, '직물': {'통합': '직물'}}

In [9]:
rs_card_data.MCT_CAT_CD = [category_integration_dict[cnt]['통합'] for cnt in rs_card_data.MCT_CAT_CD]

In [10]:
rs_card_data.MCT_CAT_CD.unique()

array(['숙박', '레저용품', '레저업소', '전기가전', '주방용구', '연료판매', '광학제품', '유통업',
       '사무통신', '자동차정비', '의료기관', '보건위생', '요식업소', '음료식품', '수리서비스', '문화취미',
       '가구', '의복', '신변잡화', '서적문구', '직물', '자동차판매'], dtype=object)

### 범주 편경, 요일 생성

In [11]:
rs_card_data.STD_DD = pd.to_datetime(rs_card_data.STD_DD,format='%Y%m%d')
rs_card_data.SEX_CD = rs_card_data.SEX_CD.astype('category')

In [12]:
rs_card_data['year'] = rs_card_data.STD_DD.dt.year
rs_card_data['month'] = rs_card_data.STD_DD.dt.month
rs_card_data['day'] = rs_card_data.STD_DD.dt.day
rs_card_data['week'] = rs_card_data.STD_DD.dt.weekofyear

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


In [13]:
rs_card_data.head()

,STD_DD,GU_CD,DONG_CD,MCT_CAT_CD,SEX_CD,AGE_CD,USE_CNT,USE_AMT,year,month,day,week
0,2019-02-01,110,517,숙박,F,25,11,595,2019,2,1,5
1,2019-02-01,110,517,숙박,F,50,5,243,2019,2,1,5
2,2019-02-01,110,517,숙박,F,55,5,27,2019,2,1,5
3,2019-02-01,110,517,숙박,M,20,11,243,2019,2,1,5
4,2019-02-01,110,517,숙박,M,25,11,433,2019,2,1,5


## 함수

In [178]:
def year_diff_by_day(data, draw_plot=True):
    ''' 전년대비 증가율 (주 버전이 아닌, 일 버전)
    data : dataframe으로 필요한 column 총 4개 STD_DD, MCT_CAT_CD, USE_AMT, year
    draw_plot : 그림 그릴지 유무
    '''
    data = data.groupby(['STD_DD', 'MCT_CAT_CD', 'year']).USE_AMT.sum().reset_index()
    data19 = data[data.year == 2019][['STD_DD','MCT_CAT_CD','USE_AMT']]
    data20 = data[data.year == 2020][['STD_DD','MCT_CAT_CD','USE_AMT']]
    data19['STD_DD'] = data19['STD_DD'] + pd.Timedelta('364 days 00:00:00') # 연도 맞춤
    table = data20.set_index(['STD_DD', 'MCT_CAT_CD']) - data19.set_index(['STD_DD', 'MCT_CAT_CD'])  # 20년 - 19년
    diff_table = table.dropna().reset_index()
    diff_table.columns = ['date', 'MCT_CAT_CD', 'diff']
    if draw_plot:
        fig = px.line(diff_table, x="date", y="diff", color='MCT_CAT_CD')
        fig.show()
    return diff_table

# 코로나

In [23]:
covid_data = pd.read_csv('../../data/covid19/covid19_confirmed.csv',dtype=str).drop('Unnamed: 0',axis=1)

In [24]:
covid_data['month'] = np.array([cnt.split('.') for cnt in covid_data.date])[:,0].astype(np.int64)
covid_data['day'] = np.array([cnt.split('.') for cnt in covid_data.date])[:,1].astype(np.int64)

In [25]:
covid_data = covid_data.melt(['month','day','date'], var_name='city', value_name='covid')

In [26]:
covid_data['date'] = pd.to_datetime(['2020'+cnt for cnt in covid_data.date],format='%Y%m.%d', infer_datetime_format=True)

In [27]:
covid_data['week'] = covid_data['date'].dt.weekofyear

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


In [28]:
covid_data.covid = covid_data.covid.astype(np.int64)

In [29]:
covid_data = covid_data.pivot_table(index=['date','city'],values='covid', aggfunc='sum').reset_index()

In [30]:
covid_data

,date,city,covid
0,2020-02-01,daegu,0
1,2020-02-01,seoul,0
2,2020-02-02,daegu,0
3,2020-02-02,seoul,0
4,2020-02-03,daegu,0
...,...,...,...
387,2020-08-12,seoul,15
388,2020-08-13,daegu,1
389,2020-08-13,seoul,26
390,2020-08-14,daegu,0


# 대구

In [179]:
gu_dict = {140:'seoul', 350:'seoul', 260:'daegu', 110:'daegu'}

rs_card_data['city'] = [gu_dict[cnt] for cnt in rs_card_data.GU_CD]

In [180]:
soeul_중구_data = rs_card_data[rs_card_data.GU_CD.isin([140])]
soeul_노원구_data = rs_card_data[rs_card_data.GU_CD.isin([350])]
daegu_수성구_data = rs_card_data[rs_card_data.GU_CD.isin([260])]
daegu_중구_data = rs_card_data[rs_card_data.GU_CD.isin([110])]

In [181]:
diff_data = year_diff_by_day(daegu_수성구_data, draw_plot=True)

In [182]:
merge_data = pd.merge(covid_data[covid_data.city=='daegu'],diff_data, on=['date'], how='right' ) 

In [184]:
merge_data

,date,city,covid,MCT_CAT_CD,diff
0,2020-02-01,daegu,0,가구,-3041.0
1,2020-02-01,daegu,0,광학제품,8837.0
2,2020-02-01,daegu,0,레저업소,34264.0
3,2020-02-01,daegu,0,레저용품,-25541.0
4,2020-02-01,daegu,0,문화취미,-39942.0
...,...,...,...,...,...
2581,2020-05-29,daegu,0,자동차정비,43596.0
2582,2020-05-29,daegu,0,자동차판매,-57185.0
2583,2020-05-29,daegu,0,전기가전,-12816.0
2584,2020-05-29,daegu,0,주방용구,2903.0


In [186]:
fig = px.scatter(merge_data, x="covid", y="diff", color='MCT_CAT_CD')
fig.show()

In [25]:
fig = px.line(merge_data, x="week", y="covid")
fig.show()

In [26]:
fig = px.line(merge_data, x="week", y="diff", color='MCT_CAT_CD')
fig.show()

# 서울

In [27]:
diff_data = year_diff(soeul_중구_data, draw_plot=False, xticks=False)
merge_data = pd.merge(covid_data[covid_data.city=='seoul'],diff_data, on=['week'], how='right' ) 

In [28]:
fig = px.scatter(merge_data, x="covid", text="week", y="diff", color='MCT_CAT_CD')
fig.show()

In [29]:
fig = px.line(merge_data, x="week", y="covid")
fig.show()

In [30]:
fig = px.line(merge_data, x="week", y="diff", color='MCT_CAT_CD')
fig.show()